# EPF using LSTM

## Importing packages

In [ ]:
!pip install torchviz

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import torch
import torch.nn as nn
from torch.autograd import Variable
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_percentage_error,r2_score, mean_absolute_error
from torchviz import make_dot

## Data processing and loading

In [ ]:
 from google.colab import drive
drive.mount('/content/drive')

In [ ]:
dir = "drive/MyDrive/Colab_Notebooks/data_"
prices_df = pd.read_csv(dir+"/prices.csv")

In [ ]:
prices_df.head()

In [ ]:
print(len(pd.isna(prices_df["prices"])==False))
print(len(prices_df["prices"]))
print(3*365*24)

In [ ]:
df_test = prices_df.iloc[:92*24,:]
print(df_test.head())
df_test.plot(x='dates', y='prices')
plt.show()

In [ ]:
l = df_test['dates'].unique()
print(len(df_test['prices'])==92*24)
prices_arr = df_test.values
print(len(prices_arr[prices_arr==0]))
print(len(l))
print(l)


In [ ]:
prices_arr = prices_df.iloc[:,2].values
print(type(prices_arr))
#plt.plot(training_set, label = 'Shampoo Sales Data')
plt.figure(figsize=(20, 6), dpi=80)
plt.plot(prices_arr, label = 'Electricity Prices')
plt.show()

### Data loading

In [ ]:
def restructering(data, input_days):
    x = []
    y = []

    day_len = 24
    num_hours = len(data)
    num_of_days = num_hours//day_len

    data_ = data.reshape(num_of_days,day_len)
    print(data_.shape)
    
    num_of_lines = len(data_)//input_days
    for i in range(num_of_days-input_days):
        _x = data_[i:(i+input_days)]
        _y = data_[i+input_days]
        x.append(_x)
        y.append(_y)

    return np.array(x),np.array(y)

In [ ]:
sc = MinMaxScaler()
training_data = sc.fit_transform(prices_arr.reshape(-1,1))

input_days = 7

x, y = restructering(training_data, input_days)

print(len(x))
train_size = int(len(y) * 0.67)
test_size = len(y) - train_size

dataX = Variable(torch.Tensor(np.array(x)))
dataY = Variable(torch.Tensor(np.array(y)))

trainX = Variable(torch.Tensor(np.array(x[0:train_size])))
trainY = Variable(torch.Tensor(np.array(y[0:train_size])))

testX = Variable(torch.Tensor(np.array(x[train_size:len(x)])))
testY = Variable(torch.Tensor(np.array(y[train_size:len(y)])))

In [ ]:
print(trainX.shape)
print(trainX.size(0))
print(test_size)

## Model

In [ ]:
class LSTM(nn.Module):

    def __init__(self, num_classes, input_size, hidden_size, num_layers):
        super(LSTM, self).__init__()
        
        self.num_classes = num_classes
        self.num_layers = num_layers
        self.input_size = input_size
        self.hidden_size = hidden_size
        #self.seq_length = seq_length
        
        self.lstm = nn.LSTM(input_size=input_size, hidden_size=hidden_size,
                            num_layers=num_layers, batch_first=True)
        
        self.fc = nn.Linear(hidden_size, num_classes)

    def forward(self, x):
        h_0 = Variable(torch.zeros(
            self.num_layers, x.size(0), self.hidden_size))
        
        c_0 = Variable(torch.zeros(
            self.num_layers, x.size(0), self.hidden_size))
        
        # Propagate input through LSTM
        ula, (h_out, _) = self.lstm(x, (h_0, c_0))
        
        h_out = h_out.view(-1, self.hidden_size)
        
        out = self.fc(h_out)
        
        return out

In [ ]:
def my_plot(epochs, loss_train, loss_val):
    plt.figure(1,figsize=(7,5))
    #ax = plt.gca()
    plt.yscale("log")
    plt.plot(epochs, loss_train)
    plt.plot(epochs,loss_val,'r')
    plt.xlabel('num of Epochs')
    plt.ylabel('loss')
    plt.title('training loss vs test loss')
    plt.grid(True)
    #ax.set_ylim([0,0.25])
    plt.legend(['train','val'])

## Training

In [ ]:
num_epochs = 3000
learning_rate = 0.003
scheduler = 0.5

input_size = 24
hidden_size = 100
num_layers = 1

num_classes = 24

lstm = LSTM(num_classes, input_size, hidden_size, num_layers)

criterion = torch.nn.MSELoss()    # mean-squared error for regression
optimizer = torch.optim.Adam(lstm.parameters(), lr=learning_rate, weight_decay=1e-5)


epoch_loss_train = []
epoch_loss_test = []
epochs = []

# Train the model
for epoch in range(1,num_epochs+1):
    outputs = lstm(trainX)
    optimizer.zero_grad()
    
    # obtain the loss function
    loss = criterion(outputs, trainY)
    
    loss.backward()
    
    optimizer.step()
    
    with torch.no_grad():
      outputs_test = lstm(testX)
      loss_test = criterion(outputs_test, testY)

    if epoch % 100 == 0:
      print("Epoch: %d, train loss: %1.5f, test loss: %1.5f" % (epoch, loss.item(), loss_test.item()))
      #print("Epoch: %d, " % (epoch, ))
      epoch_loss_train.append(loss.item())
      epoch_loss_test.append(loss_test.item())
      epochs.append(epoch)
      learning_rate = learning_rate*scheduler

In [ ]:
my_plot(np.array(epochs).astype(int), epoch_loss_train, epoch_loss_test)
plt.show()

## Evaluation

In [ ]:
"""PATH = dir+"/lstm_model"
torch.save(lstm.state_dict(), PATH)"""

In [ ]:
input_size = 24
hidden_size = 100
num_layers = 1

num_classes = 24

PATH = dir+"/lstm_model"
lstm = LSTM(num_classes, input_size, hidden_size, num_layers)
lstm.load_state_dict(torch.load(PATH))

In [ ]:
!pip install torchinfo

In [ ]:
print(testX.shape)

In [ ]:
lstm.eval()
from torchinfo import summary
from torchsummary import summary
inputs = torch.zeros((360, 7, 24))
#summary(lstm, (7,24))
for name, param in lstm.named_parameters():
    print(name)

In [ ]:
print(lstm)
pred_test = lstm(testX)
pred_train = lstm(trainX)

In [ ]:
print(pred_test.shape)
print(testY.shape)
make_dot(pred_test, params=dict(list(lstm.named_parameters()))).render("rnn_torchviz", format="png")


In [ ]:
total_mse_train = 0
total_mape_train = 0
total_mae_train = 0
total_mse_test = 0
total_mape_test = 0
total_mae_test = 0

test_len = len(testY)
train_len = len(trainY)

for i in range (test_len):
  total_mse_train += mean_squared_error(pred_train[i].detach().numpy(),trainY[i].detach().numpy())
  total_mape_train += mean_absolute_percentage_error(pred_train[i].detach().numpy(),trainY[i].detach().numpy())
  total_mae_train += mean_absolute_error(pred_train[i].detach().numpy(),trainY[i].detach().numpy())

  total_mse_test += mean_squared_error(pred_test[i].detach().numpy(),testY[i].detach().numpy())
  total_mape_test += mean_absolute_percentage_error(pred_test[i].detach().numpy(),testY[i].detach().numpy())
  total_mae_test += mean_absolute_error(pred_test[i].detach().numpy(),testY[i].detach().numpy())

print("train mse:" , total_mse_train/train_len )
print("train mape:" , total_mape_train/train_len )
print("train mae:" , total_mae_train/train_len )
print("\n")
print("test mse:" , total_mse_test/test_len )
print("test mape:" , total_mape_test/test_len )
print("test mae:" , total_mae_test/test_len )

print(test_len)

In [ ]:
plt.plot(pred_test[6].detach().numpy())
plt.plot(testY[6].detach().numpy())
plt.show()

In [ ]:
plt.figure(figsize=(20, 6), dpi=80)
fig, axs = plt.subplots(2,2)

fig.suptitle('real prices vs predicted prices')
for i in range(2):
  for j in range(2):
    axs[i, j].plot(pred_test[i+j].detach().numpy())
    axs[i, j].plot(testY[i+j].detach().numpy())



In [ ]:

fig, axs = plt.subplots(6,6)
fig.set_figheight(15)
fig.set_figwidth(15)
fig.suptitle('real prices vs predicted prices')
for i in range(6):
  for j in range(6):
    tmp_pred = sc.inverse_transform(pred_test[5+i+j].detach().numpy().reshape(-1,1))
    tmp_real = sc.inverse_transform(testY[5+i+j].detach().numpy().reshape(-1,1))
    axs[i, j].plot(tmp_pred)
    axs[i, j].plot(tmp_real)

In [ ]:
print(pred_test.shape)
lstm_pred = pred_test.detach().numpy().reshape(1,-1)[0]
test_real = testY.detach().numpy().reshape(1,-1)[0]

In [ ]:
test_real

In [ ]:
lstm_pred

In [ ]:
stats_pred_df = pd.read_csv(dir+'/Forecast_stats.csv')
stats_pred = stats_pred_df["Forecast"].values

fnn_pred_df = pd.read_csv(dir+'/Forecast_fnn.csv')
fnn_pred = fnn_pred_df["0"].values

cnn_pred_df = pd.read_csv(dir+'/Forecast_cnn.csv')
cnn_pred = cnn_pred_df["0"].values

In [ ]:
lstm_pred = sc.inverse_transform(lstm_pred.reshape(-1,1)).reshape(1,-1)[0]
stats_pred = sc.inverse_transform(stats_pred.reshape(-1,1)).reshape(1,-1)[0]
fnn_pred = sc.inverse_transform(fnn_pred.reshape(-1,1)).reshape(1,-1)[0]
cnn_pred = sc.inverse_transform(cnn_pred.reshape(-1,1)).reshape(1,-1)[0]
test_real = sc.inverse_transform(test_real.reshape(-1,1)).reshape(1,-1)[0]

In [ ]:
plt.figure(figsize=(15, 8), dpi=80)
plt.plot(stats_pred[500:1500])
plt.plot(fnn_pred[500:1500],'b')
plt.plot(test_real[500:1500],'r')
plt.show()

In [ ]:
def plotting_comparison(real_values, lstm_prediction, stats_prediction, cnn_prediction, fnn_prediction):
    fig, axs = plt.subplots(2, 2, figsize=(7,5))
    fig.set_figheight(15)
    fig.set_figwidth(15)
    duration = 200
    n1 =100
    n2 = 2000
    n3 = 4000
    n4 = 6000

    axs[0, 0].plot(stats_prediction[n1:n1+duration], "g")
    axs[0, 0].plot(lstm_prediction[n1:n1+duration],'b')

    axs[0, 0].plot(fnn_prediction[n1:n1+duration].copy(),'y')
    #axs[0, 0].plot(np.ones(duration),'y')
    axs[0, 0].plot(cnn_prediction[n1:n1+duration],'m')
    axs[0, 0].plot(real_values[n1:n1+duration],'k')

    axs[0, 0].set_xlabel('time')
    axs[0, 0].set_ylabel('prices')
    axs[0, 0].set_title('Comparison between models')
    axs[0, 0].legend(['Arima','Lstm',  'FNN', 'CNN','Real'])
    
    axs[0, 1].plot(stats_prediction[n2:n2+duration], "g")
    axs[0, 1].plot(lstm_prediction[n2:n2+duration],'b')
    axs[0, 1].plot(fnn_prediction[n2:n2+duration],'c')
    axs[0, 1].plot(cnn_prediction[n2:n2+duration],'m')
    axs[0, 1].plot(real_values[n2:n2+duration],'k')
    axs[0, 1].set_xlabel('time')
    axs[0, 1].set_ylabel('prices')
    axs[0, 1].set_title('Comparison between models')
    axs[0, 1].legend(['Arima','Lstm', 'FNN', 'CNN', 'Real'])

    axs[1, 0].plot(stats_prediction[n3:n3+duration], "g")
    axs[1, 0].plot(lstm_prediction[n3:n3+duration],'b')
    axs[1, 0].plot(fnn_prediction[n3:n3+duration],'c')
    axs[1, 0].plot(cnn_prediction[n3:n3+duration],'m')
    axs[1, 0].plot(real_values[n3:n3+duration],'k')
    axs[1, 0].set_xlabel('time')
    axs[1, 0].set_ylabel('prices')
    axs[1, 0].set_title('Comparison between models')
    axs[1, 0].legend(['Arima','Lstm', 'FNN', 'CNN', 'Real'])

    axs[1, 1].plot(stats_prediction[n4:n4+duration], "g")
    axs[1, 1].plot(lstm_prediction[n4:n4+duration],'b')
    axs[1, 1].plot(fnn_prediction[n4:n4+duration],'c')
    axs[1, 1].plot(cnn_prediction[n4:n4+duration],'m')
    axs[1, 1].plot(real_values[n4:n4+duration],'k')
    axs[1, 1].set_xlabel('time')
    axs[1, 1].set_ylabel('prices')
    axs[1, 1].set_title('Comparison between models')
    axs[1, 1].legend(['Arima','Lstm', 'FNN', 'CNN', 'Real'])





    """    plt.plot(stats_prediction[500:1200], "g")
    plt.plot(lstm_prediction[500:1200],'b')
    plt.plot(real_values[500:1200],'r')
    plt.xlabel('prices')
    plt.ylabel('time')
    plt.title('Comparison between models')
    #plt.grid(True)
    #ax.set_ylim([0,0.25])
    plt.legend(['Arima','Lstm', 'Real'])"""

In [ ]:
plotting_comparison(test_real, lstm_pred, stats_pred,cnn_pred, fnn_pred )
plt.show()

In [ ]:
plt.plot(fnn_pred[100:600])